In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
# obtain files

mingredients = pd.read_csv('meridian ingredients updated.csv')
descriptors = pd.read_csv('cancer ingredients mordred descriptors + morgan fp dropped columns.csv')
drop = []
for i in range(0, len(mingredients)):
    if mingredients.loc[i, 'IDs'] not in descriptors['IDs'].to_list():
        drop.append(i)
        
mingredients = mingredients.drop(drop).reset_index(drop=True)

In [ ]:
df_herbs = pd.read_csv('meridians updated herbs.csv')
herb_index = {}
for r in range(0, len(df_herbs)):
    herb_index[df_herbs.loc[r, 'id']] = r

In [ ]:
herb_dict = {}
for r in range(0, len(df_herbs)):
    try:
        items = df_herbs.loc[r, 'Ingredients'].split(', ')
        name = df_herbs.loc[r, 'id']
        herb_dict[name] = items
    except:
        continue

In [ ]:
index_dict = {}
for r in range(0, len(descriptors)):
    index_dict[descriptors.loc[r, 'IDs']] = r

In [ ]:
# construct dataset

table = []
results = []
for r in range(0, len(mingredients)):
    if r % 500 == 0:
        print(r)
    compound = mingredients.loc[r, 'IDs']
    index = index_dict[compound]
    table.append(descriptors.loc[index].to_list()[1:])
    herbs = []
    for i in list(herb_dict):
        if compound in herb_dict[i]:
            herbs.append(i)
    
    meridians = []
    for i in herbs:
        m = df_herbs.loc[herb_index[i], 'Meridians']
        if pd.isnull(m) is False:
            m = m.split('; ')
            for i in m:
                if i not in meridians:
                    meridians.append(i)
    temp = []
    if 'Spleen' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Stomach' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Liver' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Kidney' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Heart' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Large Intestine' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Small Intestine' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Bladder' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Gallbladder' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Cardiovascular' in meridians:
        temp.append(1)
    else:
        temp.append(0)
    if 'Three end' in meridians:
        temp.append(1)
    else:
        temp.append(0)
        
    results.append(temp)
    
file = np.array(table)
file = pd.DataFrame(file, columns=descriptors.columns.to_list()[1:])

In [ ]:
# train models

models = {}
accuracies = []
rocs = []
f1s = []
scalers = {}
test_sets = []
test_results = []
x = file

for r in range(0, 11):
    print(f'iteration: {r + 1}')
    y = [x[r] for x in results]
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    pre_save = x_test
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    for r1 in range(0, 10):
        print(f'sub iteration: {r1 + 1}')
        clf = RandomForestClassifier()
        clf.fit(x_train, y_train)
        predictions = clf.predict(x_test)
        if r1 == 0 or f1_score(y_test, predictions) > best_score:
            best_score = f1_score(y_test, predictions)
            best_accuracy = accuracy_score(y_test, predictions)
            best_roc = roc_auc_score(y_test, predictions)
            best_model = clf
            
    models[f'model{r}'] = best_model
    scalers[f'scaler{r}'] = scaler
    rocs.append(best_roc)
    accuracies.append(best_accuracy)
    f1s.append(best_score)
    test_sets.append(pre_save)
    test_results.append(y_test)

In [ ]:
# save models, scalers, and test sets

for i in range(0, len(test_sets)):
    test_sets[i].insert(0, 'Results', test_results[i])

In [ ]:
for i in range(0, len(test_sets)):
    test_sets[i].to_csv(f'meridian models/test sets/test set {i}.csv', index=False)

In [ ]:
for i in list(scalers):
    joblib.dump(scalers[i], f'meridian models/scalers/{i}.save')

In [ ]:
for i in list(models):
    joblib.dump(models[i], f'meridian models/{i}.save')